In [2]:
!pip install torch transformers datasets[audio] evaluate jiwer onnx tf2onnx tensorflow tf-keras accelerate tensorboard gradio librosa scipy

In [3]:
import torch
import time
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
print("1 Done")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
print("2 Done")

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
)

1 Done
2 Done


In [4]:
#url file
audio_file_1 = r"C:\Users\Acer\Notebook Programming\Kaizen Speaking\test_5_detik.mp3"
audio_file_2 = r"C:\Users\Acer\Notebook Programming\Kaizen Speaking\test_30_detik.mp3"

#konfigurasi spesifik bahasa yang digunakan
language = {
    "language": "indonesian"
}

start_test_1 = time.time()

result = pipe(audio_file_1, generate_kwargs=language, return_timestamps=True)
print("Hasil Timestamp:") 
print(result["chunks"])
print("\n")
print("Hasil Transcript")
print(result["text"])

end_test_1 = time.time()
duration = end_test_1 - start_test_1
print("\n")
print(f"Durasi proses : {duration:.2f} detik")

print("\n")

start_test_2 = time.time()

result = pipe(audio_file_2, generate_kwargs=language, return_timestamps=True)
print("Hasil Timestamp:")
print(result["chunks"])
print("\n")
print("Hasil Transcript")
print(result["text"])

end_test_2 = time.time()
duration = end_test_2 - start_test_2
print("\n")
print(f"Durasi proses : {duration:.2f} detik")

D:\Programming\Anaconda\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=indonesian, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=indonesian.


Hasil Timestamp:
[{'timestamp': (0.6, 4.86), 'text': ' Percobaan model speech-to-text Kaizen Speaking by Kaizen Room'}]


Hasil Transcript
 Percobaan model speech-to-text Kaizen Speaking by Kaizen Room


Durasi proses : 11.63 detik


Hasil Timestamp:
[{'timestamp': (0.0, 6.6), 'text': ' Halo, tes suara 123. Namaku Kimi Dandi Yudanarko. Aku dari Politeknik Negeri Jember.'}, {'timestamp': (6.94, 11.96), 'text': ' Dan saat ini aku sedang mengikuti Bangkit Akademi 2024 batch 2.'}, {'timestamp': (12.5, 16.16), 'text': ' Dan sedang mengikuti Path Machine Learning.'}, {'timestamp': (17.22, 25.28), 'text': ' Saat ini saya sedang belajar untuk membuat model dengan kemampuan speech-to-text recognition'}, {'timestamp': (25.28, 29.52), 'text': ' menggunakan whisper dari Hugging Face.'}, {'timestamp': (0.0, 29.98), 'text': ' Terima kasih.'}]


Hasil Transcript
 Halo, tes suara 123. Namaku Kimi Dandi Yudanarko. Aku dari Politeknik Negeri Jember. Dan saat ini aku sedang mengikuti Bangkit Akademi 2024

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "id", split="train[:10%]", trust_remote_code=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "id", split="test[:10%]", trust_remote_code=True)


In [7]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3", language="indonesian", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language="indonesian", task="transcribe")


In [8]:
from functools import partial
import torch
from scipy.signal import resample

def prepare_dataset(batch, processor):
    audio = batch["audio"]
    if audio["sampling_rate"] != 16000:
        # Resample menggunakan scipy
        number_of_samples = int(len(audio["array"]) * 16000 / audio["sampling_rate"])
        audio["array"] = resample(audio["array"], number_of_samples)
        audio["sampling_rate"] = 16000

    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

prepare_dataset_with_processor = partial(prepare_dataset, processor=processor)
common_voice = common_voice.map(prepare_dataset_with_processor, remove_columns=common_voice.column_names["train"], num_proc=4)


Map (num_proc=4):   0%|          | 0/505 [00:00<?, ? examples/s]

NameError: name 'resample' is not defined

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="indonesian", task="transcribe")
